In [303]:
import pandas as pd

# Defining custom processing function for each .tsv file
def process_tsv_file(file_path):

    #Those tvs files have info with inconsistent rows and column. But those are in 1st 10 columns.
    # So extracting those by putting custom column names
    c_names= [1,2,3,4,5,6,7,8,9,10]

    #these are tsv (tab separater values) files. To read those pd.read_csv with '\t' separator has to be used
    df = pd.read_csv(file_path, sep='\t', header=None, names= c_names)

    #As per our target file extracting 0th 1st & 3rd rows with 0th & 1st columns of values. Then transposing those to get our desired shape.
    # Then resetting index to start index from 0 & dropping existing index so that those won't be a part of dataframe of sdf1. Index needs 
    # to be reset in order to reindexing it in later step for successful concate with other dataframe.
    sdf1 = df.iloc[[0,1,3],[0,1]].T.reset_index(drop=True)

    #As per our target file extracting all values starting from 6th rows onward and all columns of values.
    # Then resetting index to start index from 0 & dropping existing index so that those won't be a part of dataframe of sdf2. Index needs 
    # to be reset in order to reindexing it in later step for successful concate with other dataframe.
    # Here we also filling the null/ NaN values with blank so it doesn't conflict with ffill() in later step.
    sdf2 = df.iloc[6:,].fillna('').reset_index(drop=True)

    #In order to concat 2 dataframes side by side perfectly both the dataframes need to have same row numbers. So here we determine the 
    # max row number to make both the dataframe equal in rows for flawless concat.
    max_rows = max(len(sdf1), len(sdf2))

    #Based on the max_rows this will create (if needed) rows and fill those with NaN value to make it equal to other dataframe.
    sdf1re = sdf1.reindex(range(max_rows))

    #Based on the max_rows this will create (if needed) rows and fill those with NaN value to make it equal to other dataframe.
    sdf2re = sdf2.reindex(range(max_rows))
    
    # Concatenating 2 dataframe to make one side by side by using axis = 1
    df3sub = pd.concat([sdf1re, sdf2re], axis= 1)

    #Forward filling the NaN/ null values with previous available value.
    df3sub = df3sub.ffill()
    
    #To make the 1st row as header/ Column name of the dataframe
    df3sub.columns = df3sub.iloc[0]
    
    #Delete that column from dataset which is made header.
    df_processed = df3sub.drop(df3sub.index[0]).reset_index(drop=True)
    return df_processed

In [304]:
#To process all tsv files individually

# Specify the .tsv files
file1 = r"E:\Python practice\tsv File convert\FBA181BC7LP4.tsv"
file2 = r"E:\Python practice\tsv File convert\FBA181BCD5HX.tsv"
file3 = r"E:\Python practice\tsv File convert\FBA181B7GSP5.tsv"

#Applying custom function on those files
file1final = process_tsv_file(file1)
file2final = process_tsv_file(file2)
file3final = process_tsv_file(file3)

In [305]:
#To created combined excel file from all tsv files putting those manually one by one
#Excel file (combined_data.xlsx) will be created in the same location of this ipynb file

with pd.ExcelWriter('combined_data.xlsx', engine='openpyxl') as writer:
    file1final.to_excel(writer, sheet_name='Sheet1', index=False, startrow=0)
    file2final.to_excel(writer, sheet_name='Sheet1', index=False, header= False, startrow=len(file1final)+1)
    file3final.to_excel(writer, sheet_name='Sheet1', index=False, header= False, startrow=(len(file1final) + len(file2final))+1)

In [296]:
#If xlsxwriter isn't installed then that has to be installed by running command pip install xlsxwriter

pip install xlsxwriter

   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/159.9 kB ? eta -:--:--
   ------- ------------------------------- 30.7/159.9 kB 325.1 kB/s eta 0:00:01
   --------- ----------------------------- 41.0/159.9 kB 326.8 kB/s eta 0:00:01
   ----------------- --------------------- 71.7/159.9 kB 393.8 kB/s eta 0:00:01
   ------------------------ ------------- 102.4/159.9 kB 420.8 kB/s eta 0:00:01
   ------------------------------- ------ 133.1/159.9 kB 522.9 kB/s eta 0:00:01
   ------------------------------------ - 153.6/159.9 kB 482.7 kB/s eta 0:00:01
   -------------------------------------- 159.9/159.9 kB 456.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [306]:
#To automatically read all tsv files from a folder apply custom defined function on those files 
# and automatically write those in a single excel file

#Excel file (output.xlsx) will be created in the same location of this ipynb file

#If xlsxwriter isn't installed then that has to be installed by running command pip install xlsxwriter

import glob
import os

# Specify the folder containing the .tsv files
folder_path = r'E:\Python practice\tsv File convert'

# Use glob to get all .tsv files in the folder
tsv_files = glob.glob(os.path.join(folder_path, "*.tsv"))

# Create an ExcelWriter object to write to the same sheet
with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
    start_row = 0  # Initialize row counter for writing data to Excel
    
    for i, file in enumerate(tsv_files):
        # Process each file using the custom function
        df_processed = process_tsv_file(file)
        
        # Write the first DataFrame with headers
        if i == 0:
            df_processed.to_excel(writer, sheet_name='Sheet1', startrow=start_row, index=False)
            start_row += len(df_processed) + 1  # Update start_row for next DataFrame
        
        # Write the rest of the DataFrames without headers
        else:
            df_processed.to_excel(writer, sheet_name='Sheet1', startrow=start_row, index=False, header=False)
            start_row += len(df_processed)   # Update start_row for the next DataFrame